# kNN - k Nearest Neightbours

É um algoritmo do tipo *lazy*, isto é, não gera um modelo para os dados de treinamento, apenas os memoriza e, dada uma instância de teste, calcula-se a distância para cada instância do conjunto de treino e, pelo voto da maioria dos **k vizinhos mais próximos**, define-se a classe.

### Distância de Minkowski
Calcula-se a Distância de Minkowski $\textbf{d}$ fazendo $$ d(\textbf{x}^{(i)},\textbf{x}^{(j)})=\left(\sum_{k}|x_{k}^{(i)}-x_{k}^{(j)}|^{p}\right)^{\frac{1}{p}}, $$ em que:
* $\textbf{x}^{(i)}$ é uma instância dos dados de teste, cujos elementos são $x_{k}^{(i)}$;
* $\textbf{x}^{(j)}$ é uma instância dos dados de treino, cujos elementos são $x_{k}^{(j)}$;
* $p$ é a ordem da distância.

### Algoritmo

1. Definir a métrica de distância;
2. Dado uma instância de teste, calcular a distância para cada instância de treino e armazená-las em uma lista;
3. Ordenar a lista com as distâncias e selecionar os $\textbf{k}$ elementos com menor distância;
4. Nesses $\textbf{k}$ elementos, ver a classe de maior frequência para atribuí-la à instância teste. Em caso de empate, atribui-se a classe referente à menor distância.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
def dist(x_test, x_train, p):
    """
    Implementa a Distância de Minkowski:
    Recebe dois vetores x_teste e x_train e retorna a distância entre eles.
    """
    
    sum_ = 0
    
    #Soma o módulo diferença elevada a p dos elementos dos vetores
    for i in range(len(x_test)):
        sum_ += abs(x_test[i] - x_train[i])**p
    
    #retorna a raíz p-ésima da soma
    return sum_**(1/p)


In [3]:
class KNearestNeighbours:
    def __init__(self, x_train, y_train):
        
        #if not isinstance(x_train, np.ndarray):
        self.x_train = np.array(x_train)
            
        #if not isinstance(y_train, np.ndarray):
        self.y_train = np.array(y_train)

            
    def predict(self, x_test, y_test, k, p):
        
        self.x_test = x_test
        self.y_test = y_test
        
        if not isinstance(self.x_test, np.ndarray):
            self.x_test = np.array(x_test)
            
        if not isinstance(self.y_test, np.ndarray):
            self.y_test = np.array(self.y_test)
            
        self.p = p
        self.k = k

        # número de exemplos de teste
        n_test = self.x_test.shape[0]
        
        yhat = []
        t = 0

        for i in range(n_test):
            
            distance_list = [dist(self.x_test[i], self.x_train[j], self.p)
                             for j in range(self.x_train.shape[0])]
            
            distances = pd.DataFrame({'distance': distance_list,
                                      'label': self.y_train})
            
            distances = distances.sort_values(by=['distance'])[:self.k]
            
            #majority vote
            nn_labels_counter = Counter(self.y_train[distances.index])
            pred = nn_labels_counter.most_common()[0][0]
            
            print(t)
            t += 1
            yhat.append(pred)
        
        return yhat

In [4]:
path = '~/ML-AZ/census.csv'
df = pd.read_csv(path)

In [5]:
df.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

#OneHotEncoder cria uma coluna dummy para cada atributo nominal único, de
#modo a não criar uma ordem ao atribuir labels do tipo 0, 1, 3, 4...
#ColumnTransformer transforma seletivamente as colunas de um array multi-atributos
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [1,3,5,6,7,8,9,13])],remainder='passthrough')
x = onehotencorder.fit_transform(x).toarray()
print(x.shape)

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)
print(y.shape)

(32561, 108)
(32561,)


In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25,
                                                    random_state=0)

print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

x_train shape:  (24420, 108)
x_test shape:  (8141, 108)
y_train shape:  (24420,)
y_test shape:  (8141,)


In [9]:
clf = KNearestNeighbours(x_train, y_train)

In [10]:
predictions = clf.predict(x_test, y_test, 5, 2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62


KeyboardInterrupt: 